In [104]:
import pandas as pd 
import os
import requests
import re
from zipfile import ZipFile
from bs4 import BeautifulSoup


# This script downloads PUMD diary files from an arbitrary year from the BLS website. It extracts the .csv from the .zip, returns a list of unique values from certain columns, and then resaves the .csv file as a new .zip file.

# Set the year to search for in the last cell of the notebook, and then pass that into the search_bls() function


In [105]:
def download_file(file_url, file_name):
    """ Downloads selected zip file from BLS 

        Keyword Arguments:
        file_url -- URL of file to download
        file_name -- name of downloaded file

        Returns:
        zip_file_path -- path to downloaded zip file

     """
    r = requests.get(file_url)

    if r.status_code == 200:
        print(f'Downloading file: {file_name} ')
        with open(file_name, "wb") as zip:
            zip.write(r.content)

        if os.path.isfile(file_name):
            print(f'Success! File downloaded: {file_name}.')
            zip_file_path = file_name
        else:
            print(f'Error! File not downloaded: {file_name}.')
            zip_file_path = ''
    else:
        print(f'Error. Invalid URL/file: {file_url}.')
        zip_file_path = ''

    return zip_file_path


In [106]:
def extract_csv(zip_file_path):
    """ Extracts specific csv file from zip_file 

        Keyword Arguments:
        zip_file -- name of zip archive

        Returns:
        expd_file_path - text string of csv path & filename

     """
    # list to hold all expd... files
    expd_list = []

    # check if zip_file_path isn't empty and if it exists in PWD
    if zip_file_path != '' and os.path.isfile(zip_file_path): 

        with ZipFile(zip_file_path, 'r') as zip:
            # loop through all members of the archive
            for filemember in zip.namelist():
                # if expd is in the file name, then applend that name to the expd list
                if 'expd' in filemember:
                    expd_list.append(filemember)
            #sort expd_list in ascending order, just in case the .zip file wasn't saved in ascending order 
            expd_list.sort()

            #get first file from list
            first_expd_file = expd_list[0] 

            # extract to PWD
            zip.extract(first_expd_file)

        if os.path.isfile(first_expd_file):
            print(f'Success: File Extracted: {first_expd_file}')
            expd_file_path = first_expd_file
        else:
            print(f'Error in Extraction. {first_expd_file} not extracted.')
            expd_file_path = ''
    else:
        print(f'Error in Extraction. {zip_file_path} not available.')
    
    #return textstring of extractedfile  
    return expd_file_path






In [107]:
def analyze_csv(csv_file_path):
    """ analyze csv file in  csv_file_path

        Keyword Arguments:
        csv_file_path -- file path to csv file to analyze
     """

    # check if csv_file_path isn't empty and if it exists in PWD
    if csv_file_path != '' and os.path.isfile(csv_file_path): 

        # begin analyis
        df = pd.read_csv(csv_file_path)
        unique_ids = list(df['NEWID'].unique())

        # report analysis
        print('Analysis Complete')
        print(f'There are {len(unique_ids)} unique ids in {csv_file_path}. They are:')
        print(unique_ids)

    else:
        print(f'Error analyzing CSV. {csv_file_path} not available.')

In [123]:
def save_new_zip(csv_file_path):
    """ saves new zip archive from file in csvfile_path

        Keyword Arguments:
        csv_file_path -- file path to csv file
     """
    
    # check if csv_file_path isn't empty and if it exists in PWD
    if csv_file_path != '' and os.path.isfile(csv_file_path): 
        # make new file name

        # new_file_name = re.sub(r'^.*?/', '/', csv_file_path) + '.zip'
        pos_last_slash = csv_file_path.rfind('/')
        print(pos_last_slash)
        new_file_name = csv_file_path[pos_last_slash:] + '.zip'
        print(new_file_name)
    #     # create new zip archive
    #     with ZipFile(new_file_name,'w') as zip:
    #         zip.write(csv_file_path)
        
    #     # test for successful file creation
    #     if os.path.isfile(new_file_name):
    #         print(f'Success: CSV saved to new zip archive: {new_file_name}')
    #     else:
    #         print(f'Error creating new archive. {new_file_name} created.')
    
    # else:
        print(f'Error creating new archive. {csv_file_path} not available.')

        
        
    

In [124]:
def search_bls(year):
    """ searches the BLS for PUMD Diary files for the Year that is passed into the function 

        Keyword Arguments:
        year -- The year to search for 
     """

    #  set url variables for bls site and PUMD html page
    bls_base_url = 'https://www.bls.gov'
    pumd_url = bls_base_url + '/cex/pumd_data.htm'

    # try to scrape BLS pumd page
    r = requests.get(pumd_url)

    if r.status_code == 200:

        #get soup object to scrape page
        soup = BeautifulSoup(r.content, 'lxml')

        #  diary links in csv format all take this form: '/cex/pumd/data/comma/diaryYY.zip'
        #  so we parse the year argument and create a file to look for, and then create a test_url to find the file on the page. 
        zip_file = 'diary' + str(year)[2:4] + '.zip'
        test_url =  '/cex/pumd/data/comma/' + zip_file

        diary_link = soup.find(href=test_url)
        #  if we find an available link, we download the zip and extract it. 
        if(diary_link):
            download_link = bls_base_url + diary_link['href']
            zip_file_path = download_file(download_link,zip_file)
            csv_to_analyze = extract_csv(zip_file_path)
            analyze_csv(csv_to_analyze)
            save_new_zip(csv_to_analyze)

        #  if link isn't found, report to the user that the selected year is not available. 
        else:
            print(f'PUMD Diary for {year} is not available')

    #  if pumd_url isn't valid, report to the user that the pumd_url is not available. 
    else:
        print(f'Error! {pumd_url} not valid.')


In [125]:
# Set the Year variable in this cell, and run the search_bls function from here with the selected year. 

year = 2008

search_bls(year)

02, 893611, 893612, 893621, 893622, 893631, 893632, 893641, 893642, 893651, 893652, 893661, 893662, 893671, 893672, 893681, 893682, 893691, 893692, 893701, 893702, 893712, 893721, 893722, 893741, 893742, 893751, 893752, 893761, 893762, 893771, 893772, 893781, 893782, 893791, 893792, 893801, 893802, 893811, 893812, 893831, 893832, 893841, 893842, 893851, 893852, 893861, 893862, 893872, 893881, 893882, 893891, 893892, 893901, 893902, 893921, 893922, 893931, 893932, 893941, 893942, 893961, 893962, 893972, 893981, 893982, 893991, 893992, 894001, 894002, 894011, 894012, 894021, 894022, 894031, 894032, 894041, 894042, 894051, 894052, 894061, 894062, 894072, 894081, 894082, 894091, 894092, 894101, 894102, 894111, 894112, 894121, 894122, 894131, 894132, 894141, 894142, 894151, 894152, 894161, 894162, 894171, 894172, 894181, 894182, 894191, 894192, 894201, 894202, 894212, 894231, 894232, 894241, 894242, 894251, 894252, 894261, 894262, 894271, 894272, 894281, 894282, 894291, 894292, 894301, 8943

FileNotFoundError: [Errno 2] No such file or directory: './expd081.csvz/expd081.csvi/expd081.csvp'